<a href="https://colab.research.google.com/github/lucywowen/csci191_ProgSci/blob/main/hackathons/conditional_loop_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipulating Data Hackathon

Ok first things first... let's assign y'all to groups!

In [5]:
import random

# Set the random seed for reproducibility
random.seed(4)

students = [
    "Adam", "Andrew", "Cameron", "Liam",
    "Lindsay", "Mason", "Solange", "Simon"
]

random.shuffle(students) # Randomly shuffles the list of students in place

# Create groups of 2
for i in range(0, len(students), 2):
    group = students[i:i+2]
    print(f"Group: {group}")

Group: ['Andrew', 'Solange']
Group: ['Mason', 'Lindsay']
Group: ['Simon', 'Adam']
Group: ['Cameron', 'Liam']


Ok now let's unzip in the data.  

In [1]:
# --- Setup ---
import os, glob, zipfile
from pathlib import Path
import pandas as pd

BASE = Path("hackathon_dataset")
ZIP  = Path("hackathon_dataset.zip")

# Unzip if needed
if not BASE.exists() and ZIP.exists():
    import zipfile
    with zipfile.ZipFile(ZIP, "r") as zf:
        zf.extractall("")

print("Data folder exists:", BASE.exists())


Data folder exists: True


## Task 1 — Load & Combine

Ok, your first task is to load and combine the data.  We want to:

1) Read all 'measurements_2024-??.csv' and handle schema drift with simple if statements
2) Add 'source_file', and concat with union of columns

In [6]:
# --- Task 1: Load & Combine (union of columns, source_file) ---
# Read all 'measurements_2024-??.csv', handle schema drift with simple if statements,
# add 'source_file', and concat with union of columns.

## Here's a hint: you can use `if "String" in df.columns` and a conditional!
import pandas as pd
from pathlib import Path

data_folder = BASE / "data"
files = sorted((data_folder).glob(" ")) ## fill this in
all_dfs = []

for f in files:
    df_month = pd.read_csv(f)

    # TO DO: Handle schema drift: tempC -> temp_c


    # TO DO: Handle schema drift: sensorID -> sensor_id


    # TO DO: Add provenance column


    all_dfs.append(df_month)

# Union-concat: sort=False keeps all columns encountered across files
df_all = pd.concat(all_dfs, ignore_index=True, sort=False)
print("Combined shape:", df_all.shape)
df_all.head()


Combined shape: (2443, 7)


,date,sensor_id,temp_c,status,source_file,notes,misc_flag
0,2024-01-15,S022,0.91,WARN,measurements_2024-01.csv,NaN,NaN
1,2024-01-11,S002,2.52,OK,measurements_2024-01.csv,NaN,NaN
2,2024-01-08,NaN,5.86,ERROR,measurements_2024-01.csv,NaN,NaN
3,2024-01-21,S013,-1.79,ERROR,measurements_2024-01.csv,NaN,NaN
4,2024-01-07,NaN,5.85,WARN,measurements_2024-01.csv,NaN,NaN


## Task 2 — Normalize + Filter + Flag Suspicious Temps

Ok now that we've done that, let's create a create 'status_norm' with loops and basic string methods.  I'll provide you that code!

Then let's drop rows where status missing/empty or equals 'error' (case-insensitive).

Also, let's create 'suspicious_temp' when `temp_c < -30 or > 50` (row-wise with loops).

In [ ]:
# --- Task 2: Normalize 'status' and filter + flag suspicious temps ---
# We'll create 'status_norm' with loops and basic string methods.
# Then: drop rows where status missing/empty or equals 'error' (case-insensitive).
# Also create 'suspicious_temp' when temp_c < -30 or > 50 (row-wise with loops).

import math

# 1) Normalize status -> 'status_norm'
status_norm = []
for i in range(len(df_all)):
    val = df_all.loc[i, "status"] if "status" in df_all.columns else ""
    # treat NaN as empty string
    if pd.isna(val):
        val = ""
    val = str(val).strip().lower()
    status_norm.append(val)

df_all["status_norm"] = status_norm

# 2) Flag suspicious temps
suspicious = []
for i in range(len(df_all)):
    if "temp_c" in df_all.columns:
        t = df_all.loc[i, "temp_c"]
        # handle NaN
        if pd.isna(t):
            suspicious.append(False)
        else:
            # TO DO: YOU FILL THIS IN!
    else:
        suspicious.append(False)

df_all["suspicious_temp"] = suspicious

# 3) Filter out missing/empty status OR 'error'
keep = []
for i in range(len(df_all)):
    s = df_all.loc[i, "status_norm"]
    if # TO DO: FILL THIS IN
        keep.append(True)
    else:
        keep.append(False)

df_clean = df_all.loc[keep].copy()
print("After status cleaning:", df_clean.shape)
df_clean.head()


## Task 3 — Validate Rows (loops only)

Ok now let's try to make sure all the values are there... I'll provide the code for `is_data` (and it's a try statement! that's new!) as well as `is_sensor_ok`, but you do the rest!

In [ ]:
# --- Task 3: Validation ---
# We create: is_date, is_sensor_ok, is_temp_ok, is_status_ok, then 'is_valid' with loops.

# 1) is_date: try parsing each, if fails -> False
is_date = []
for i in range(len(df_clean)):
    try:
        pd.to_datetime(df_clean.loc[i, "date"], errors="raise")
        is_date.append(True)
    except Exception:
        is_date.append(False)

# 2) is_sensor_ok: non-empty string in 'sensor_id'
is_sensor_ok = []
for i in range(len(df_clean)):
    if "sensor_id" in df_clean.columns:
        sid = df_clean.loc[i, "sensor_id"]
    else:
        sid = ""
    if pd.isna(sid):
        is_sensor_ok.append(False)
    else:
        sid = str(sid)
        if len(sid) > 0:
            is_sensor_ok.append(True)
        else:
            is_sensor_ok.append(False)

# 3) is_temp_ok: -50 <= temp_c <= 60 (NaN -> False)
is_temp_ok = []

# TO DO: YOUR TURN!

# 4) is_status_ok: in {'OK','WARN','ERROR'} (case-insensitive check using status_norm)
is_status_ok = []

# TO DO: YOU TRY!

df_clean["is_date"] = is_date
df_clean["is_sensor_ok"] = is_sensor_ok
df_clean["is_temp_ok"] = is_temp_ok
df_clean["is_status_ok"] = is_status_ok

# 5) is_valid = all four True
is_valid = []
for i in range(len(df_clean)):
    if df_clean.loc[i, "is_date"] and df_clean.loc[i, "is_sensor_ok"] and df_clean.loc[i, "is_temp_ok"] and df_clean.loc[i, "is_status_ok"]:
        is_valid.append(True)
    else:
        is_valid.append(False)

df_clean["is_valid"] = is_valid

print("Valid fraction:", round(float(sum(is_valid))/len(is_valid), 3))
df_clean[["is_date","is_sensor_ok","is_temp_ok","is_status_ok","is_valid"]].head()


## Task 4 — Logs Parsing

Ok now for the last thing, you try the whole thing again but on the `logs` data.  So your goal is to count how many rows per month have status 'ok' (case-insensitive, strip spaces).

You got this!!

In [ ]:
# --- Task 4: Logs parsing (months 01..09) with loops ---
# Goal: count how many rows per month have status 'ok' (case-insensitive, strip spaces).

# TO DO: Find how many rows per month have the status ok

ok_counts = pd.DataFrame({"month": months, "n_ok": n_ok})
ok_counts


## Task 5 — Save Outputs

And finally, save that out!  You'll submit (in canvas) a `measurements_2024_clean_basic.csv` and `logs_ok_counts_basic.csv' (one for each group).

In [ ]:
# --- Task 5: Save outputs ---
out_dir = BASE / "outputs_basic"
out_dir.mkdir(exist_ok=True)
df_clean.to_csv(out_dir/"measurements_2024_clean_basic.csv", index=False)
ok_counts.to_csv(out_dir/"logs_ok_counts_basic.csv", index=False)
print("Wrote outputs to:", out_dir)
